In [1]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
cfns = pd.read_csv('../data/classifications_cset.csv')

In [3]:
cfns.sample(5)

,classifications.Annotator,classifications.Annotation Status,classifications.Reviewer,classifications.Quality Control,classifications.Full Description,classifications.Short Description,classifications.Beginning Date,classifications.Ending Date,classifications.Location,classifications.Near Miss,...,classifications.Sector of Deployment,classifications.Public Sector Deployment,classifications.Nature of End User,classifications.Level of Autonomy,classifications.Relevant AI functions,classifications.AI Techniques,classifications.AI Applications,classifications.Physical System,classifications.Problem Nature,classifications.Publish
41,5,6. Complete and final,5,False,"From 1982 to 1986, St George's Hospital Medica...","From 1982 to 1986, St George's Hospital Medica...",1982-01-01T00:00:00.000Z,1986-01-01T00:00:00.000Z,"London, England",Harm caused,...,Human health and social work activities,False,Amateur,Medium,"[""Cognition""]",Machine learning,"[""decision support""]","[""Software only""]","[""Specification""]",True
79,5,6. Complete and final,5,False,Scottish soccer team Inverness Caledonian This...,In a Scottish soccer match the AI-enabled ball...,2020-10-24T07:00:00.000Z,2020-10-24T07:00:00.000Z,"Inverness, Scotland, UK",Harm caused,...,"Arts, entertainment and recreation",False,Amateur,High,"[""Perception"",""Cognition"",""Action""]",supervised learning,"[""image classification"",""image recognition""]","[""Consumer device""]","[""Robustness""]",True
51,6,6. Complete and final,6,False,"On Jun 6, 2016, 18 year-old Kabir Alli pointed...","On June 6, 2016, Google image searches of ""thr...",2016-06-06T07:00:00.000Z,2016-06-06T07:00:00.000Z,Global,Unclear/unknown,...,Information and communication,False,Amateur,High,"[""Perception"",""Cognition"",""Action""]","open-source, machine learning","[""image classification"",""search engine"",""conte...","[""Software only""]","[""Robustness"",""Assurance""]",True
94,5,4. Peer review complete,6,False,"On May 4, 2017, a U.S. federal judge advanced ...","On May 4, 2017, a U.S. federal judge advanced ...",2011-01-01T00:00:00.000Z,2016-01-01T00:00:00.000Z,"Houston, TX",Harm caused,...,Education,True,Amateur,Low,"[""Cognition""]",machine learning,"[""data analytics"",""decision support""]","[""Software only""]","[""Specification""]",False
55,5,4. Peer review complete,6,False,"In July 2016, Australia’s Department of Human ...","In July 2016, Australia’s Department of Human ...",7/2016,2017-01-01T00:00:00.000Z,Australia,Harm caused,...,Public administration and defence,True,Amateur,High,"[""Cognition"",""Action""]",Unclear,"[""decision support""]","[""Software only""]","[""Unknown/unclear""]",False


In [4]:
# clean up!
cfns.insert(0, 'incident', cfns.index + 1)
cfns.columns = cfns.columns.str.lower().str.replace('classifications.', '').str.replace(' ', '_')
cfns['beginning_date'] = pd.to_datetime(cfns['beginning_date'], utc=True).dt.date
cfns['ending_date'] = pd.to_datetime(cfns['ending_date'], utc=True).dt.date

def parse_list_columns(column):
    if type(column[0]) == str and column[0].startswith('['):
        return column.apply(eval)
    else: return column

cfns = cfns.apply(parse_list_columns)

In [ ]:
# cfns.to_csv('../data/classifications_processed.csv', index=False)

In [5]:
cfns.columns

Index(['annotator', 'annotation_status', 'reviewer', 'quality_control',
       'full_description', 'short_description', 'beginning_date',
       'ending_date', 'location', 'near_miss', 'named_entities',
       'technology_purveyor', 'intent', 'severity', 'lives_lost',
       'harm_distribution_basis', 'harm_type', 'infrastructure_sectors',
       'financial_cost', 'laws_implicated', 'ai_system_description',
       'data_inputs', 'system_developer', 'sector_of_deployment',
       'public_sector_deployment', 'nature_of_end_user', 'level_of_autonomy',
       'relevant_ai_functions', 'ai_techniques', 'ai_applications',
       'physical_system', 'problem_nature', 'publish'],
      dtype='object')

In [6]:
cfns.shape

(96, 33)

In [7]:
cfns[cfns['publish'] == True].shape

(63, 33)

In [8]:
# harms: multiple options
cfns['harm_type'].apply(pd.Series).stack().value_counts()

Harm to social or political systems                           18
Harm to civil liberties                                       17
Harm to physical health/safety                                17
Psychological harm                                            17
Financial harm                                                 9
Harm to physical property                                      8
Harm to intangible property                                    3
Other:Privacy                                                  2
Other:Reputational harm/social harm (libel and defamation)     1
Other:Unable to detect facial features                         1
Other:Reputational harm                                        1
Other:Harm to publicly available information                   1
Other:Reputational harm; False incarceration                   1
dtype: int64

In [9]:
cfns['technology_purveyor'].apply(pd.Series).stack().value_counts()

Google                                       18
Amazon                                        6
Microsoft                                     4
Tesla                                         4
Facebook                                      4
                                             ..
Instant Checkmate                             1
Nicos Isaak                                   1
Oakland Police Department                     1
The Decentralized Autonomous Organization     1
Stanford Medical Center                       1
Length: 83, dtype: int64

In [10]:
cfns.columns

Index(['annotator', 'annotation_status', 'reviewer', 'quality_control',
       'full_description', 'short_description', 'beginning_date',
       'ending_date', 'location', 'near_miss', 'named_entities',
       'technology_purveyor', 'intent', 'severity', 'lives_lost',
       'harm_distribution_basis', 'harm_type', 'infrastructure_sectors',
       'financial_cost', 'laws_implicated', 'ai_system_description',
       'data_inputs', 'system_developer', 'sector_of_deployment',
       'public_sector_deployment', 'nature_of_end_user', 'level_of_autonomy',
       'relevant_ai_functions', 'ai_techniques', 'ai_applications',
       'physical_system', 'problem_nature', 'publish'],
      dtype='object')

In [11]:
for i in cfns.columns:
    u = cfns[i].unique()
    if len(u) < 10:
        print(i, u)
    else: print(i)

annotator [1 2 3 4 6 5]
annotation_status ['6. Complete and final' '3. In peer review' '4. Peer review complete']
reviewer [5 6 2 3]
quality_control [False  True]
full_description
short_description
beginning_date
ending_date
location
near_miss ['Harm caused' 'Unclear/unknown' 'Near miss']


TypeError: unhashable type: 'list'

In [ ]:
cfns.head(1)

,classifications.Annotator,classifications.Annotation Status,classifications.Reviewer,classifications.Quality Control,classifications.Full Description,classifications.Short Description,classifications.Beginning Date,classifications.Ending Date,classifications.Location,classifications.Near Miss,...,classifications.Sector of Deployment,classifications.Public Sector Deployment,classifications.Nature of End User,classifications.Level of Autonomy,classifications.Relevant AI functions,classifications.AI Techniques,classifications.AI Applications,classifications.Physical System,classifications.Problem Nature,classifications.Publish
0,1,6. Complete and final,5,False,"On December 5, 2018, a robot punctured a can o...",Twenty-four Amazon workers in New Jersey were ...,2018-12-05T08:00:00.000Z,2018-12-05T08:00:00.000Z,"Robbinsville, NJ",Harm caused,...,Transportation and storage,False,NaN,Unclear/unknown,"[""Unclear""]",NaN,"[""robotics""]","[""Unknown/unclear""]","[""Unknown/unclear""]",True


In [12]:
s = cfns['classifications.Harm Type'].value_counts()
q = np.percentile(s, 50)
s[s > q].index

KeyError: 'classifications.Harm Type'

In [13]:
s = cfns['classifications.Harm Type'].apply(eval).apply(pd.Series).stack().value_counts()
q = np.percentile(s, 50)
s[s > q].index

KeyError: 'classifications.Harm Type'

In [14]:
s

NameError: name 's' is not defined

In [15]:
cfns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   annotator                 96 non-null     int64 
 1   annotation_status         96 non-null     object
 2   reviewer                  96 non-null     int64 
 3   quality_control           96 non-null     bool  
 4   full_description          96 non-null     object
 5   short_description         96 non-null     object
 6   beginning_date            95 non-null     object
 7   ending_date               94 non-null     object
 8   location                  79 non-null     object
 9   near_miss                 96 non-null     object
 10  named_entities            96 non-null     object
 11  technology_purveyor       96 non-null     object
 12  intent                    96 non-null     object
 13  severity                  96 non-null     object
 14  lives_lost                96